# PySpark_GraphFrames_AirportDataset

## PySpark 实践指南 第7章

英文书名是《Learning PySpark》

这里是本书的github：https://github.com/drabastomek/learningPySpark

本文件直接copy自原书提供的代码。

原始地址
* Original blog post: [On-Time Flight Performance with GraphFrames with Apache Spark Blog Post](https://databricks.com/blog/2016/03/16/on-time-flight-performance-with-graphframes-for-apache-spark.html)
* Original Notebook: [On-Time Flight Performance with GraphFrames with Apache Spark Notebook](http://cdn2.hubspot.net/hubfs/438089/notebooks/Samples/Miscellaneous/On-Time_Flight_Performance.html)

`departuredelays.csv`
该数据集包含美国的航空公司报告的计划、实际的起飞和到达时间以及航班延误原因

`airport-codes-na.txt`
此数据集包含美国机场数据的列表，包括IATA代码、机场名称和机场位置

使用命令上传到HDFS

这里能加载这些数据集并创建相应的Spark DataFrame。在此我们可以轻松地推断这两个文件的模式（schema）

一旦我们加载了departureDelays DataFrame，我们可以把它加入缓存以便用高效的方式来对数据做一些额外的过滤

### Preparation
Extract the Airports and Departure Delays information from S3 / DBFS

使用如下命令就能直接从相应目录导入相应数据，似乎是直接存在S3 / DBFS目录下的。

In [4]:
%fs ls /databricks-datasets/flights/

path,name,size
dbfs:/databricks-datasets/flights/README.md,README.md,412
dbfs:/databricks-datasets/flights/airport-codes-na.txt,airport-codes-na.txt,11411
dbfs:/databricks-datasets/flights/departuredelays.csv,departuredelays.csv,33396236


### cache() 方法
Spark 中一个很重要的能力是将数据持久化（或称为缓存），在多个操作间都可以访问这些持久化的数据。当持久化一个 RDD 时，每个节点的其它分区都可以使用 RDD 在内存中进行计算，在该数据上的其他 action 操作将直接使用内存中的数据。这样会让以后的 action 操作计算速度加快（通常运行速度会加速 10 倍）。缓存是迭代算法和快速的交互式使用的重要工具。

RDD 可以使用 persist() 方法或 cache() 方法进行持久化。数据将会在第一次 action 操作时进行计算，并缓存在节点的内存中。Spark 的缓存具有容错机制，如果一个缓存的 RDD 的某个分区丢失了，Spark 将按照原来的计算过程，自动重新计算并进行缓存。

DataFrame的cache和persist的区别。通过源码，可以看到cache()依然调用的persist()，但是persist调用cacheQuery，而cacheQuery的默认存储级别为MEMORY_AND_DISK，这点和rdd是不一样的。

- MEMORY_ONLY : 将 RDD 以反序列化 Java 对象的形式存储在 JVM 中。如果内存空间不够，部分数据分区将不再缓存，在每次需要用到这些数据时重新进行计算。这是默认的级别。
- MEMORY_AND_DISK : 将 RDD 以反序列化 Java 对象的形式存储在 JVM 中。如果内存空间不够，将未缓存的数据分区存储到磁盘，在需要使用这些分区时从磁盘读取。

In [6]:
# Set File Paths
tripdelaysFilePath = "/databricks-datasets/flights/departuredelays.csv"
airportsnaFilePath = "/databricks-datasets/flights/airport-codes-na.txt"

# Obtain airports dataset
airportsna = spark.read.csv(airportsnaFilePath, header='true', inferSchema='true', sep='\t')
airportsna.createOrReplaceTempView("airports_na")

# Obtain departure Delays data
departureDelays = spark.read.csv(tripdelaysFilePath, header='true')
departureDelays.createOrReplaceTempView("departureDelays")
# 持久化
departureDelays.cache()

# Available IATA codes from the departuredelays sample dataset
# 的查询允许我们用出发城市IATA代码（例如Seattle='SEA'，San Francisco='SFO'，New York JFK='JFK'等）构建一个没有重复的列表。
tripIATA = spark.sql("select distinct iata from (select distinct origin as iata from departureDelays union all select distinct destination as iata from departureDelays) a")
tripIATA.createOrReplaceTempView("tripIATA")

# Only include airports with atleast one trip from the departureDelays dataset
# 接下来，我们只需要收集在于departureDelays DataFrame中保留记录的时间段内有航班起飞过的机场：
airports = spark.sql("select f.IATA, f.City, f.State, f.Country from airports_na f join tripIATA t on t.IATA = f.IATA")
airports.createOrReplaceTempView("airports")
airports.cache()

Out[ 7 ]: DataFrame[IATA: string, City: string, State: string, Country: string]

In [7]:
departureDelays.count()

Out[ 8 ]: 1391578

In [8]:
# 通过构建无重复的起飞机场代码的列表，我们可以构建airports DataFrame，它只包含departureDelays数据集中存在的机场代码。以下代码段生成了一个新的DataFrame（departureDelays_geo），它由一些关键属性，包括航班日期、延迟、距离和机场信息（出发地、目的地）组成：

departureDelays_geo = spark.sql("select cast(f.date as int) as tripid, \
cast(concat(concat(concat(concat(concat(concat('2014-', concat(concat(substr(cast(f.date as string), 1, 2), '-')), substr(cast(f.date as string), 3, 2)), ' '), substr(cast(f.date as string), 5, 2)), ':'), substr(cast(f.date as string), 7, 2)), ':00') as timestamp) as `localdate`, \
cast(f.delay as int), \ 
cast(f.distance as int), \ 
f.origin as src, \ 
f.destination as dst, \
o.city as city_src, \
d.city as city_dst, \
o.state as state_src, \
d.state as state_dst \
from departuredelays f join airports o on o.iata = f.origin join airports d on d.iata = f.destination") 

# Create Temporary View and cache
departureDelays_geo.createOrReplaceTempView("departureDelays_geo")
departureDelays_geo.cache()

# Count
departureDelays_geo.count()

Out[ 9 ]: 1361141

In [9]:
# Review the top 10 rows of the `departureDelays_geo` DataFrame
departureDelays_geo.show(10)

+-------+-------------------+-----+--------+---+---+-----------+-------------------+---------+---------+
 tripid| localdate|delay|distance|src|dst| city_src| city_dst|state_src|state_dst|
+-------+-------------------+-----+--------+---+---+-----------+-------------------+---------+---------+
1011111|2014-01-01 11:11:00| -5| 221|MSP|INL|Minneapolis|International Falls| MN| MN|
1021111|2014-01-02 11:11:00| 7| 221|MSP|INL|Minneapolis|International Falls| MN| MN|
1031111|2014-01-03 11:11:00| 0| 221|MSP|INL|Minneapolis|International Falls| MN| MN|
1041925|2014-01-04 19:25:00| 0| 221|MSP|INL|Minneapolis|International Falls| MN| MN|
1061115|2014-01-06 11:15:00| 33| 221|MSP|INL|Minneapolis|International Falls| MN| MN|
1071115|2014-01-07 11:15:00| 23| 221|MSP|INL|Minneapolis|International Falls| MN| MN|
1081115|2014-01-08 11:15:00| -9| 221|MSP|INL|Minneapolis|International Falls| MN| MN|
1091115|2014-01-09 11:15:00| 11| 221|MSP|INL|Minneapolis|International Falls| MN| MN|
1101115|2014-01-10 11:15:00| -3| 221|MSP|INL|Minneapolis|International Falls| MN| MN|
1112015|2014-01-11 20:15:00| -7| 221|MSP|INL|Minneapolis|International Falls| MN| MN|
+-------+-------------------+-----+--------+---+---+-----------+-------------------+---------+---------+
only showing top 10 rows

In [10]:
# Using `display` to view the data
display(departureDelays_geo)

tripid,localdate,delay,distance,src,dst,city_src,city_dst,state_src,state_dst
1011111,2014-01-01 11:11:00,-5,221,MSP,INL,Minneapolis,International Falls,MN,MN
1021111,2014-01-02 11:11:00,7,221,MSP,INL,Minneapolis,International Falls,MN,MN
1031111,2014-01-03 11:11:00,0,221,MSP,INL,Minneapolis,International Falls,MN,MN
1041925,2014-01-04 19:25:00,0,221,MSP,INL,Minneapolis,International Falls,MN,MN
1061115,2014-01-06 11:15:00,33,221,MSP,INL,Minneapolis,International Falls,MN,MN
1071115,2014-01-07 11:15:00,23,221,MSP,INL,Minneapolis,International Falls,MN,MN
1081115,2014-01-08 11:15:00,-9,221,MSP,INL,Minneapolis,International Falls,MN,MN
1091115,2014-01-09 11:15:00,11,221,MSP,INL,Minneapolis,International Falls,MN,MN
1101115,2014-01-10 11:15:00,-3,221,MSP,INL,Minneapolis,International Falls,MN,MN
1112015,2014-01-11 20:15:00,-7,221,MSP,INL,Minneapolis,International Falls,MN,MN


## Building the Graph
现在我们已经导入了自己的数据，接下来让我们来构建自己的图形。为此，我们将为我们自己的节点和边构建结构。在撰写本文时，GraphFrames需要对点和边的命名做特定的约定：
* 表示顶点的列需要id的名称。在我们的例子中，我们的航班数据的节点是机场。因此，我们需要在airports DataFrame中将IATA机场代码重新命名为id。
* 表示边的列需要一个起点（src）和终点（dst）。对于我们的航班数据，边是航班，因此src和dst是departureDelays_geo DataFrame中的出发地和目的地的列中的数据。

**WARNING:** If the graphframes package, required in the cell below, is not installed, follow the instructions [here](http://cdn2.hubspot.net/hubfs/438089/notebooks/help/Setup_graphframes_package.html).

In [13]:
# Note, ensure you have already installed the GraphFrames spack-package
from pyspark.sql.functions import *
from graphframes import *

# Create Vertices (airports) and Edges (flights)
tripVertices = airports.withColumnRenamed("IATA", "id").distinct()
tripEdges = departureDelays_geo.select("tripid", "delay", "src", "dst", "city_dst", "state_dst")

# Cache Vertices and Edges
tripEdges.cache()
tripVertices.cache()

Out[ 12 ]: DataFrame[id: string, City: string, State: string, Country: string]

In [14]:
# Vertices
#   The vertices of our graph are the airports
display(tripVertices)

id,City,State,Country
FAT,Fresno,CA,USA
CMH,Columbus,OH,USA
PHX,Phoenix,AZ,USA
PAH,Paducah,KY,USA
COS,Colorado Springs,CO,USA
MYR,Myrtle Beach,SC,USA
RNO,Reno,NV,USA
SRQ,Sarasota,FL,USA
VLD,Valdosta,GA,USA
PSC,Pasco,WA,USA


In [15]:
# Edges
#  The edges of our graph are the flights between airports
display(tripEdges)

tripid,delay,src,dst,city_dst,state_dst
1011111,-5,MSP,INL,International Falls,MN
1021111,7,MSP,INL,International Falls,MN
1031111,0,MSP,INL,International Falls,MN
1041925,0,MSP,INL,International Falls,MN
1061115,33,MSP,INL,International Falls,MN
1071115,23,MSP,INL,International Falls,MN
1081115,-9,MSP,INL,International Falls,MN
1091115,11,MSP,INL,International Falls,MN
1101115,-3,MSP,INL,International Falls,MN
1112015,-7,MSP,INL,International Falls,MN


In [16]:
# Build `tripGraph` GraphFrame
#  This GraphFrame builds up on the vertices and edges based on our trips (flights)
tripGraph = GraphFrame(tripVertices, tripEdges)
print tripGraph

# Build `tripGraphPrime` GraphFrame
#   This graphframe contains a smaller subset of data to make it easier to display motifs and subgraphs (below)
tripEdgesPrime = departureDelays_geo.select("tripid", "delay", "src", "dst")
tripGraphPrime = GraphFrame(tripVertices, tripEdgesPrime)

GraphFrame(v:[id: string, City: string ... 2 more fields], e:[src: string, dst: string ... 4 more fields])

## Simple Queries
Let's start with a set of simple graph queries to understand flight performance and departure delays

#### Determine the number of airports and trips

In [19]:
print "Airports: %d" % tripGraph.vertices.count()
print "Trips: %d" % tripGraph.edges.count()


Airports: 279
Trips: 1361141

#### Determining the longest delay in this dataset

要确定数据集中航班最长的延迟时间，你可以运行以下查询，结果为1642分钟（超过27小时！）：

In [22]:
tripGraph.edges.groupBy().max("delay").show()

+----------+
max(delay)|
+----------+
 1642|
+----------+

In [23]:
# Finding the longest Delay
longestDelay = tripGraph.edges.groupBy().max("delay")
display(longestDelay)

max(delay)
1642


#### Determining the number of delayed vs. on-time / early flights

In [25]:
# Determining number of on-time(准时) / early flights vs. delayed flights
print "On-time / Early Flights: %d" % tripGraph.edges.filter("delay <= 0").count()
print "Delayed Flights: %d" % tripGraph.edges.filter("delay > 0").count()

On-time / Early Flights: 780469
Delayed Flights: 580672

#### What flights departing SEA are most likely to have significant delays
Note, delay can be <= 0 meaning the flight left on time or early

从以下结果可以看出，从西雅图出发的航班出现延误的前五名城市是Philadelphia（PHL）、Colorado Springs（COS）、Fresno（FAT）、Long Beach（LGB）和Washington D.C（IAD）：

In [28]:
tripGraph.edges\
  .filter("src = 'SEA' and delay > 0")\
  .groupBy("src", "dst")\
  .avg("delay")\
  .sort(desc("avg(delay)"))\
  .show(5)
  

+---+---+------------------+
src|dst| avg(delay)|
+---+---+------------------+
SEA|PHL|55.666666666666664|
SEA|COS| 43.53846153846154|
SEA|FAT| 43.03846153846154|
SEA|LGB| 39.39705882352941|
SEA|IAD|37.733333333333334|
+---+---+------------------+
only showing top 5 rows

In [29]:
display(tripGraph.edges.filter("src = 'SEA' and delay > 0").groupBy("src", "dst").avg("delay").sort(desc("avg(delay)")))

src,dst,avg(delay)
SEA,PHL,55.666666666666664
SEA,COS,43.53846153846154
SEA,FAT,43.03846153846154
SEA,LGB,39.39705882352941
SEA,IAD,37.733333333333334
SEA,MIA,37.325581395348834
SEA,SFO,36.50210378681627
SEA,SBA,36.48275862068966
SEA,JFK,35.03125
SEA,ORD,33.60335195530726


#### What destinations tend to have delays

In [31]:
# After displaying tripDelays, use Plot Options to set `state_dst` as a Key.
tripDelays = tripGraph.edges.filter("delay > 0")
display(tripDelays)

tripid,delay,src,dst,city_dst,state_dst
1021111,7,MSP,INL,International Falls,MN
1061115,33,MSP,INL,International Falls,MN
1071115,23,MSP,INL,International Falls,MN
1091115,11,MSP,INL,International Falls,MN
1171115,4,MSP,INL,International Falls,MN
2091925,1,MSP,INL,International Falls,MN
2152015,16,MSP,INL,International Falls,MN
2161925,169,MSP,INL,International Falls,MN
2171115,27,MSP,INL,International Falls,MN
2181115,96,MSP,INL,International Falls,MN


#### What destinations tend to have significant delays departing from SEA

让我们来看一下从西雅图出发到哪个州的航班有最长的累积延误（单次延误>100分钟）。这一次我们将使用display命令查看数据：

In [34]:
# States with the longest cumulative delays (with individual delays > 100 minutes) (origin: Seattle)
display(tripGraph.edges.filter("src = 'SEA' and delay > 100"))

tripid,delay,src,dst,city_dst,state_dst
3201938,108,SEA,BUR,Burbank,CA
3201655,107,SEA,SNA,Orange County,CA
1011950,123,SEA,OAK,Oakland,CA
1021950,194,SEA,OAK,Oakland,CA
1021615,317,SEA,OAK,Oakland,CA
1021755,385,SEA,OAK,Oakland,CA
1031950,283,SEA,OAK,Oakland,CA
1031615,364,SEA,OAK,Oakland,CA
1031325,130,SEA,OAK,Oakland,CA
1061755,107,SEA,OAK,Oakland,CA


## Vertex Degrees
* `inDegrees`: Incoming connections to the airport
* `outDegrees`: Outgoing connections from the airport 
* `degrees`: Total connections to and from the airport

在图论中，节点的度数是该节点周围的边数。在我们的示例中，度数是到节点（即机场）的边的总数（即航班）。因此，如果我们从图中获得最大的20个节点度数（按降序排列），那么我们将从图中得到前20个最繁忙的机场（航班进出的总数最多）。这可以使用以下查询快速得到：

In [36]:
# Degrees
#  The number of degrees - the number of incoming and outgoing connections - for various airports within this sample dataset
display(tripGraph.degrees.sort(desc("degree")).limit(20))

id,degree
ATL,179774
DFW,133966
ORD,125405
LAX,106853
DEN,103699
IAH,85685
PHX,79672
SFO,77635
LAS,66101
CLT,56103


In [37]:
# inDegrees （到达该机场的航班）
#  The number of degrees - the number of incoming connections - for various airports within this sample dataset
display(tripGraph.inDegrees.sort(desc("inDegree")).limit(20))

id,inDegree
ATL,89633
DFW,65767
ORD,61654
LAX,53184
DEN,50738
IAH,42512
PHX,39619
SFO,38641
LAS,32994
CLT,28044


In [38]:
# outDegrees（从该机场出发的航班）
#  The number of degrees - the number of outgoing connections - for various airports within this sample dataset
display(tripGraph.outDegrees.sort(desc("outDegree")).limit(20))

id,outDegree
ATL,90141
DFW,68199
ORD,63751
LAX,53669
DEN,52961
IAH,43173
PHX,40053
SFO,38994
LAS,33107
CLT,28059


## City / Flight Relationships through Motif Finding
为了更容易理解城市机场和航班之间的复杂关系，我们使用motifs命令进一步挖掘机场（例如节点）和航班（也就是边）之间的关系。DataFrame结果中的列名通过Motif key给出。请注意，Motif查找是作为GraphFrames的一部分被支持的新图形算法之一。
例如，我们来确定由旧金山国际机场（SFO）造成的延误：

#### What delays might we blame on SFO

分解下面的查询，（x）表示节点（即机场），而[xy]表示边（即机场之间的航班）。因此，为了确定SFO造成的延误，可以使用以下定义：
- 顶点（b）表示中间的机场（即SFO）；
- 顶点（a）表示出发地机场（数据集内）；
- 顶点（c）表示目的地机场（数据集内）；
- 边[ab]表示（a）（即出发地）和（b）（即SFO）之间的航班；
- 边[bc]表示（b）（即SFO）和（c）（即目的地）之间的航班。

在filter命令的声明中，我们提供了一些基本的约束（注意，这只是一个简单的航班路径表示）：
- b.id='SFO'表示中间节点（b）仅限于SFO机场；
- （ab.delay>500 or bc.delay>500）表示我们仅限于延误大于500分钟的航班；
- （bc.tripid>ab.tripid and bc.tripid< li="">

通过使用Motif查找，你可以轻松搜索图形中的结构模式。通过GraphFrames，你可以使用DataFrame的强大功能和速度来分发和执行查询。

In [42]:
# Using tripGraphPrime to more easily display 
#   - The associated edge (ab, bc) relationships 
#   - With the different the city / airports (a, b, c) where SFO is the connecting city (b)
#   - Ensuring that flight ab (i.e. the flight to SFO) occured before flight bc (i.e. flight leaving SFO)
#   - Note, TripID was generated based on time in the format of MMDDHHMM converted to int
#       - Therefore bc.tripid < ab.tripid + 10000 means the second flight (bc) occured within approx a day of the first flight (ab)
# Note: In reality, we would need to be more careful to link trips ab and bc.
motifs = tripGraphPrime.find("(a)-[ab]->(b); (b)-[bc]->(c)")\
  .filter("(b.id = 'SFO') and (ab.delay > 500 or bc.delay > 500) and bc.tripid > ab.tripid and bc.tripid < ab.tripid + 10000")
display(motifs)

a,ab,b,bc,c
"List(MDW, Chicago, IL, USA)","List(1020840, 509, MDW, SFO)","List(SFO, San Francisco, CA, USA)","List(1021802, 7, SFO, MSY)","List(MSY, New Orleans, LA, USA)"
"List(HNL, Honolulu, Oahu, HI, USA)","List(2141405, 586, HNL, SFO)","List(SFO, San Francisco, CA, USA)","List(2141638, 24, SFO, MSY)","List(MSY, New Orleans, LA, USA)"
"List(MCI, Kansas City, MO, USA)","List(3160600, 637, MCI, SFO)","List(SFO, San Francisco, CA, USA)","List(3161828, -3, SFO, MSY)","List(MSY, New Orleans, LA, USA)"
"List(MDW, Chicago, IL, USA)","List(1020840, 509, MDW, SFO)","List(SFO, San Francisco, CA, USA)","List(1021912, 12, SFO, BUR)","List(BUR, Burbank, CA, USA)"
"List(MDW, Chicago, IL, USA)","List(1020840, 509, MDW, SFO)","List(SFO, San Francisco, CA, USA)","List(1021331, 0, SFO, BUR)","List(BUR, Burbank, CA, USA)"
"List(MDW, Chicago, IL, USA)","List(1020840, 509, MDW, SFO)","List(SFO, San Francisco, CA, USA)","List(1021618, 0, SFO, BUR)","List(BUR, Burbank, CA, USA)"
"List(MDW, Chicago, IL, USA)","List(1020840, 509, MDW, SFO)","List(SFO, San Francisco, CA, USA)","List(1022019, -5, SFO, BUR)","List(BUR, Burbank, CA, USA)"
"List(MDW, Chicago, IL, USA)","List(1020840, 509, MDW, SFO)","List(SFO, San Francisco, CA, USA)","List(1030755, -1, SFO, BUR)","List(BUR, Burbank, CA, USA)"
"List(HNL, Honolulu, Oahu, HI, USA)","List(2040725, 622, HNL, SFO)","List(SFO, San Francisco, CA, USA)","List(2041828, -3, SFO, BUR)","List(BUR, Burbank, CA, USA)"
"List(HNL, Honolulu, Oahu, HI, USA)","List(2040725, 622, HNL, SFO)","List(SFO, San Francisco, CA, USA)","List(2040845, -8, SFO, BUR)","List(BUR, Burbank, CA, USA)"


## Determining Airport Ranking using PageRank
因为GraphFrames建立在GraphX之上，所以有几个算法是我们可以立即利用的。PageRank在Google Search Engine中广泛使用，由Larry Page创建。这里我们来引用Wikipedia的解释：

“PageRank的工作原理是对到连接页面的数量和质量进行计数，从而估计该页面的重要性。缺省的假定是：越是重要的网站接收到的其他网站的链接就越多。”

虽然上面的例子是关于网页的，但这一极好的理念可以用于任何图结构，无论它是来自网页、自行车站还是机场。并且GraphFrames的界面就像调用一个方法一样简单。GraphFrames.PageRank将把PageRank结果作为新的column追加到DataFrame的节点中来使我们后续的分析更加简单。

由于本数据集中包含的各机场有很多航班和连接，我们可以使用PageRank算法使Spark迭代地遍历图形，以计算出每个机场重要性的粗略估计值。

在机场排名方面，PageRank算法已经确定ATL（Hartsfield-Jackson Atlanta International Airport）是美国最重要的机场。这个观察结果是有道理的，因为ATL不仅是美国最繁忙的机场[http://bit.ly/2eTGHs4](http://bit.ly/2eTGHs4)，而且也是世界上最繁忙的机场（2000～2015）[http://bit.ly/2eTGDsy](http://bit.ly/2eTGDsy)

In [44]:
# Determining Airport ranking of importance using `pageRank`
# 请注意，resetProbability=0.15表示复位到随机节点的概率（这是默认值），而maxIter=5是设定的迭代次数。
ranks = tripGraph.pageRank(resetProbability=0.15, maxIter=5)

# 原书limit设置是20.这里不一致
display(ranks.vertices.orderBy(ranks.vertices.pagerank.desc()).limit(10))

id,City,State,Country,pagerank
ATL,Atlanta,GA,USA,18.91010461672981
DFW,Dallas,TX,USA,13.699227467378963
ORD,Chicago,IL,USA,13.163049993795983
DEN,Denver,CO,USA,9.72338828381156
LAX,Los Angeles,CA,USA,8.703656827807164
IAH,Houston,TX,USA,7.991324463091126
SFO,San Francisco,CA,USA,6.903242998287931
PHX,Phoenix,AZ,USA,6.505886984498641
SLC,Salt Lake City,UT,USA,5.799587684561127
LAS,Las Vegas,NV,USA,5.253592445609149


## Most popular flights (single city hops) 确定最受欢迎的直飞航班
Using the `tripGraph`, we can quickly determine what are the most popular single city hop flights

In [46]:
# Determine the most popular flights (single city hops)
import pyspark.sql.functions as func
topTrips = tripGraph \
  .edges \
  .groupBy("src", "dst") \
  .agg(func.count("delay").alias("trips")) 

从这个查询可以看出，直达航班中最频繁的是LAX（洛杉矶）和SFO（旧金山）之间。这些航班如此频繁的事实上表明了它们在航空市场上的重要性。如“纽约时报”2016年4月4日的文章《Alaska Air Sees Virgin America as Key to West Coast》[http://nyti.ms/2ea1uZR](http://nyti.ms/2ea1uZR)所述，在这两个机场获得机位是阿拉斯加航空公司收购Virgin航空公司的原因。图表不仅仅是有趣的，而且还包含着强大的潜在商业洞察力！

In [48]:
# Show the top 20 most popular flights (single city hops)
display(topTrips.orderBy(topTrips.trips.desc()).limit(10))

src,dst,trips
SFO,LAX,3232
LAX,SFO,3198
LAS,LAX,3016
LAX,LAS,2964
JFK,LAX,2720
LAX,JFK,2719
ATL,LGA,2501
LGA,ATL,2500
LAX,PHX,2394
PHX,LAX,2387


## 确定最大的中转机场
1.理解机场节点度的一个扩展是确定最大的中转机场。许多机场被用作中转站而不是目的地。一个简单的计算方法是通过计算inDegrees（到达该机场的航班数量）/outDegrees（离开该机场的航班数量）的比率。接近1的值可能表示大量的中转航班，而值<1表示出站航班较多，值>1表示入站航班较多。

请注意，这是一个简单的计算，不考虑航班的定时或计划，只是考虑数据集中的总数：

In [50]:
# Calculate the inDeg (flights into the airport) and outDeg (flights leaving the airport)
inDeg = tripGraph.inDegrees
outDeg = tripGraph.outDegrees

# Calculate the degreeRatio (inDeg/outDeg)
degreeRatio = inDeg.join(outDeg, inDeg.id == outDeg.id) \
  .drop(outDeg.id) \
  .selectExpr("id", "double(inDegree)/double(outDegree) as degreeRatio") \
  .cache()

# Join back to the `airports` DataFrame (instead of registering temp table as above)
nonTransferAirports = degreeRatio.join(airports, degreeRatio.id == airports.IATA) \
  .selectExpr("id", "city", "degreeRatio") \
  .filter("degreeRatio < .9 or degreeRatio > 1.1")

# List out the city airports which have abnormal degree ratios.
display(nonTransferAirports)

id,city,degreeRatio
GFK,Grand Forks,1.3333333333333333
FAI,Fairbanks,1.1232686980609419
OME,Nome,0.5084745762711864
BRW,Barrow,0.28651685393258425


如下查询是有道理的，因为这些机场是航空公司的主要枢纽（例如，达美航空使用明尼阿波利斯和盐湖城作为枢纽；Frontier使用丹佛；美国航空使用达拉斯和凤凰城；美联航使用休斯顿、芝加哥和旧金山；夏威夷航空公司以Kahului和檀香山为枢纽）。

In [52]:
# Join back to the `airports` DataFrame (instead of registering temp table as above)
transferAirports = degreeRatio.join(airports, degreeRatio.id == airports.IATA) \
  .selectExpr("id", "city", "degreeRatio") \
  .filter("degreeRatio between 0.9 and 1.1")
  
# List out the top 10 transfer city airports
# 此查询的输出是前十名中转城市机场（即枢纽机场）的条形图：
display(transferAirports.orderBy("degreeRatio").limit(10))

id,city,degreeRatio
MSP,Minneapolis,0.9375183338222353
DEN,Denver,0.958025717037065
DFW,Dallas,0.964339653074092
ORD,Chicago,0.9671063983310065
SLC,Salt Lake City,0.9827417906368358
IAH,Houston,0.9846895050147083
PHX,Phoenix,0.9891643572266746
OGG,"Kahului, Maui",0.9898718478710211
HNL,"Honolulu, Oahu",0.990535889872173
SFO,San Francisco,0.9909473252295224


## Breadth First Search 
广度优先搜索（BFS）是一种新的算法，它作为GraphFrames的一部分，可以找出从一组节点到另一组节点的最短路径。在本节中，我们将使用BFS遍历tripGraph命令来快速找到所需的节点（即机场）和边（即航班）。让我们来尝试根据数据集找到城市之间最短的连接数。请注意，这些例子不考虑时间或距离，只是在城市之间跳来跳去。例如，要查找西雅图和旧金山之间的直达航班数量，你可以运行以下查询：

In [54]:
# Example 1: Direct Seattle to San Francisco 
filteredPaths = tripGraph.bfs(
  fromExpr = "id = 'SEA'",
  toExpr = "id = 'SFO'",
  maxPathLength = 1)
display(filteredPaths)

from,e0,to
"List(SEA, Seattle, WA, USA)","List(1010710, 31, SEA, SFO, San Francisco, CA)","List(SFO, San Francisco, CA, USA)"
"List(SEA, Seattle, WA, USA)","List(1012125, -4, SEA, SFO, San Francisco, CA)","List(SFO, San Francisco, CA, USA)"
"List(SEA, Seattle, WA, USA)","List(1011840, -5, SEA, SFO, San Francisco, CA)","List(SFO, San Francisco, CA, USA)"
"List(SEA, Seattle, WA, USA)","List(1010610, -4, SEA, SFO, San Francisco, CA)","List(SFO, San Francisco, CA, USA)"
"List(SEA, Seattle, WA, USA)","List(1011230, -2, SEA, SFO, San Francisco, CA)","List(SFO, San Francisco, CA, USA)"
"List(SEA, Seattle, WA, USA)","List(1010955, -6, SEA, SFO, San Francisco, CA)","List(SFO, San Francisco, CA, USA)"
"List(SEA, Seattle, WA, USA)","List(1011100, 2, SEA, SFO, San Francisco, CA)","List(SFO, San Francisco, CA, USA)"
"List(SEA, Seattle, WA, USA)","List(1011405, 0, SEA, SFO, San Francisco, CA)","List(SFO, San Francisco, CA, USA)"
"List(SEA, Seattle, WA, USA)","List(1020710, -1, SEA, SFO, San Francisco, CA)","List(SFO, San Francisco, CA, USA)"
"List(SEA, Seattle, WA, USA)","List(1022125, -4, SEA, SFO, San Francisco, CA)","List(SFO, San Francisco, CA, USA)"


As you can see, there are a number of direct flights between Seattle and San Francisco.

In [56]:
# Example 2: Direct San Francisco and Buffalo
filteredPaths = tripGraph.bfs(
  fromExpr = "id = 'SFO'",
  toExpr = "id = 'BUF'",
  maxPathLength = 1)
display(filteredPaths)

id,City,State,Country


But there are no direct flights between San Francisco and Buffalo.

但是，如果我们想确定旧金山和布法罗之间的直飞航班数量呢？你会注意到运行上面的查询将没有结果，也就是说，两个城市之间没有直飞航班：

In [58]:
# Example 2a: Flying from San Francisco to Buffalo
# 一旦我们将前面的查询修改为maxPathLength=2，也就是一次中转，那么你将会看到更多的航班选项：
filteredPaths = tripGraph.bfs(
  fromExpr = "id = 'SFO'",
  toExpr = "id = 'BUF'",
  maxPathLength = 2)
display(filteredPaths)

from,e0,v1,e1,to
"List(SFO, San Francisco, CA, USA)","List(1010700, 0, SFO, BOS, Boston, MA)","List(BOS, Boston, MA, USA)","List(1010635, -6, BOS, BUF, Buffalo, NY)","List(BUF, Buffalo, NY, USA)"
"List(SFO, San Francisco, CA, USA)","List(1010700, 0, SFO, BOS, Boston, MA)","List(BOS, Boston, MA, USA)","List(1011059, 13, BOS, BUF, Buffalo, NY)","List(BUF, Buffalo, NY, USA)"
"List(SFO, San Francisco, CA, USA)","List(1010700, 0, SFO, BOS, Boston, MA)","List(BOS, Boston, MA, USA)","List(1011427, 19, BOS, BUF, Buffalo, NY)","List(BUF, Buffalo, NY, USA)"
"List(SFO, San Francisco, CA, USA)","List(1010700, 0, SFO, BOS, Boston, MA)","List(BOS, Boston, MA, USA)","List(1020635, -4, BOS, BUF, Buffalo, NY)","List(BUF, Buffalo, NY, USA)"
"List(SFO, San Francisco, CA, USA)","List(1010700, 0, SFO, BOS, Boston, MA)","List(BOS, Boston, MA, USA)","List(1021059, 0, BOS, BUF, Buffalo, NY)","List(BUF, Buffalo, NY, USA)"
"List(SFO, San Francisco, CA, USA)","List(1010700, 0, SFO, BOS, Boston, MA)","List(BOS, Boston, MA, USA)","List(1021427, 194, BOS, BUF, Buffalo, NY)","List(BUF, Buffalo, NY, USA)"
"List(SFO, San Francisco, CA, USA)","List(1010700, 0, SFO, BOS, Boston, MA)","List(BOS, Boston, MA, USA)","List(1030635, 0, BOS, BUF, Buffalo, NY)","List(BUF, Buffalo, NY, USA)"
"List(SFO, San Francisco, CA, USA)","List(1010700, 0, SFO, BOS, Boston, MA)","List(BOS, Boston, MA, USA)","List(1031059, 0, BOS, BUF, Buffalo, NY)","List(BUF, Buffalo, NY, USA)"
"List(SFO, San Francisco, CA, USA)","List(1010700, 0, SFO, BOS, Boston, MA)","List(BOS, Boston, MA, USA)","List(1031427, 0, BOS, BUF, Buffalo, NY)","List(BUF, Buffalo, NY, USA)"
"List(SFO, San Francisco, CA, USA)","List(1010700, 0, SFO, BOS, Boston, MA)","List(BOS, Boston, MA, USA)","List(1040635, 16, BOS, BUF, Buffalo, NY)","List(BUF, Buffalo, NY, USA)"


But there are flights from San Francisco to Buffalo with Minneapolis as the transfer point.  But what are the most popular layovers between `SFO` and `BUF`?

In [60]:
# Display most popular layover cities by descending count
display(filteredPaths.groupBy("v1.id", "v1.City").count().orderBy(desc("count")).limit(10))

id,City,count


## Loading the D3 Visualization
Using the airports D3 visualization to visualize airports and flight paths

为了获得这个数据集中航班路径和联系的强大而有趣的可视化结果，我们可以利用Databricks notebook中的Airports D3 visualization（https://mbostock.github.io/d3/talk/20111116/airports.html ）方法。通过连接我们的GraphFrames、DataFrame和D3 visualization，我们可以看到所有在数据集中被标记为准点或提前出发的航班联系范围。

蓝色圆圈表示节点（即机场），其中圆圈的大小表示进出这些机场的边（即航班）的数量。黑线是边本身（即航班）及其各自与其他节点（即机场）的连接。注意连到画面以外的边代表夏威夷和阿拉斯加州的节点（即机场）


为了这个工作，我们首先要创建一个名为d3a的scala包嵌入到我们的notebook中（你可以从这里下载：http://bit.ly/2kPkXkc ）。因为我们正在使用Databricks notebook，我们可以在我们的PySpark notebook中调用Scala：

In [62]:
%scala
package d3a
// We use a package object so that we can define top level classes like Edge that need to be used in other cells

import org.apache.spark.sql._
import com.databricks.backend.daemon.driver.EnhancedRDDFunctions.displayHTML

case class Edge(src: String, dest: String, count: Long)

case class Node(name: String)
case class Link(source: Int, target: Int, value: Long)
case class Graph(nodes: Seq[Node], links: Seq[Link])

object graphs {
val sqlContext = SQLContext.getOrCreate(org.apache.spark.SparkContext.getOrCreate())
import sqlContext.implicits._

def force(clicks: Dataset[Edge], height: Int = 100, width: Int = 960): Unit = {
  val data = clicks.collect()
  val nodes = (data.map(_.src) ++ data.map(_.dest)).map(_.replaceAll("_", " ")).toSet.toSeq.map(Node)
  val links = data.map { t =>
    Link(nodes.indexWhere(_.name == t.src.replaceAll("_", " ")), nodes.indexWhere(_.name == t.dest.replaceAll("_", " ")), t.count / 20 + 1)
  }
  showGraph(height, width, Seq(Graph(nodes, links)).toDF().toJSON.first())
}
/**
 * Displays a force directed graph using d3
 * input: {"nodes": [{"name": "..."}], "links": [{"source": 1, "target": 2, "value": 0}]}
 */
def showGraph(height: Int, width: Int, graph: String): Unit = {

displayHTML(s"""<!DOCTYPE html>
<html>
  <head>
    <link type="text/css" rel="stylesheet" href="https://mbostock.github.io/d3/talk/20111116/style.css"/>
    <style type="text/css">
      #states path {
        fill: #ccc;
        stroke: #fff;
      }

      path.arc {
        pointer-events: none;
        fill: none;
        stroke: #000;
        display: none;
      }

      path.cell {
        fill: none;
        pointer-events: all;
      }

      circle {
        fill: steelblue;
        fill-opacity: .8;
        stroke: #fff;
      }

      #cells.voronoi path.cell {
        stroke: brown;
      }

      #cells g:hover path.arc {
        display: inherit;
      }
    </style>
  </head>
  <body>
    <script src="https://mbostock.github.io/d3/talk/20111116/d3/d3.js"></script>
    <script src="https://mbostock.github.io/d3/talk/20111116/d3/d3.csv.js"></script>
    <script src="https://mbostock.github.io/d3/talk/20111116/d3/d3.geo.js"></script>
    <script src="https://mbostock.github.io/d3/talk/20111116/d3/d3.geom.js"></script>
    <script>
      var graph = $graph;
      var w = $width;
      var h = $height;

      var linksByOrigin = {};
      var countByAirport = {};
      var locationByAirport = {};
      var positions = [];

      var projection = d3.geo.azimuthal()
          .mode("equidistant")
          .origin([-98, 38])
          .scale(1400)
          .translate([640, 360]);

      var path = d3.geo.path()
          .projection(projection);

      var svg = d3.select("body")
          .insert("svg:svg", "h2")
          .attr("width", w)
          .attr("height", h);

      var states = svg.append("svg:g")
          .attr("id", "states");

      var circles = svg.append("svg:g")
          .attr("id", "circles");

      var cells = svg.append("svg:g")
          .attr("id", "cells");

      var arc = d3.geo.greatArc()
          .source(function(d) { return locationByAirport[d.source]; })
          .target(function(d) { return locationByAirport[d.target]; });

      d3.select("input[type=checkbox]").on("change", function() {
        cells.classed("voronoi", this.checked);
      });

      // Draw US map.
      d3.json("https://mbostock.github.io/d3/talk/20111116/us-states.json", function(collection) {
        states.selectAll("path")
          .data(collection.features)
          .enter().append("svg:path")
          .attr("d", path);
      });

      // Parse links
      graph.links.forEach(function(link) {
        var origin = graph.nodes[link.source].name;
        var destination = graph.nodes[link.target].name;

        var links = linksByOrigin[origin] || (linksByOrigin[origin] = []);
        links.push({ source: origin, target: destination });

        countByAirport[origin] = (countByAirport[origin] || 0) + 1;
        countByAirport[destination] = (countByAirport[destination] || 0) + 1;
      });

      d3.csv("https://mbostock.github.io/d3/talk/20111116/airports.csv", function(data) {

        // Build list of airports.
        var airports = graph.nodes.map(function(node) {
          return data.find(function(airport) {
            if (airport.iata === node.name) {
              var location = [+airport.longitude, +airport.latitude];
              locationByAirport[airport.iata] = location;
              positions.push(projection(location));

              return true;
            } else {
              return false;
            }
          });
        });

        // Compute the Voronoi diagram of airports' projected positions.
        var polygons = d3.geom.voronoi(positions);

        var g = cells.selectAll("g")
            .data(airports)
          .enter().append("svg:g");

        g.append("svg:path")
            .attr("class", "cell")
            .attr("d", function(d, i) { return "M" + polygons[i].join("L") + "Z"; })
            .on("mouseover", function(d, i) { d3.select("h2 span").text(d.name); });

        g.selectAll("path.arc")
            .data(function(d) { return linksByOrigin[d.iata] || []; })
          .enter().append("svg:path")
            .attr("class", "arc")
            .attr("d", function(d) { return path(arc(d)); });

        circles.selectAll("circle")
            .data(airports)
            .enter().append("svg:circle")
            .attr("cx", function(d, i) { return positions[i][0]; })
            .attr("cy", function(d, i) { return positions[i][1]; })
            .attr("r", function(d, i) { return Math.sqrt(countByAirport[d.iata]); })
            .sort(function(a, b) { return countByAirport[b.iata] - countByAirport[a.iata]; });
      });
    </script>
  </body>
</html>""")
  }

  def help() = {
displayHTML("""
<p>
Produces a force-directed graph given a collection of edges of the following form:</br>
<tt><font color="#a71d5d">case class</font> <font color="#795da3">Edge</font>(<font color="#ed6a43">src</font>: <font color="#a71d5d">String</font>, <font color="#ed6a43">dest</font>: <font color="#a71d5d">String</font>, <font color="#ed6a43">count</font>: <font color="#a71d5d">Long</font>)</tt>
</p>
<p>Usage:<br/>
<tt>%scala</tt></br>
<tt><font color="#a71d5d">import</font> <font color="#ed6a43">d3._</font></tt><br/>
<tt><font color="#795da3">graphs.force</font>(</br>
&nbsp;&nbsp;<font color="#ed6a43">height</font> = <font color="#795da3">500</font>,<br/>
&nbsp;&nbsp;<font color="#ed6a43">width</font> = <font color="#795da3">500</font>,<br/>
&nbsp;&nbsp;<font color="#ed6a43">clicks</font>: <font color="#795da3">Dataset</font>[<font color="#795da3">Edge</font>])</tt>
</p>""")
  }
}



Warning: classes defined within packages cannot be redefined without a cluster restart.
Compilation successful.
<notebook>:15: warning: method getOrCreate in object SQLContext is deprecated: Use SparkSession.builder instead
val sqlContext = SQLContext.getOrCreate(org.apache.spark.SparkContext.getOrCreate())
 ^

In [63]:
%scala d3a.graphs.help()

Produces a force-directed graph given a collection of edges of the following form: 
 case class Edge ( src : String , dest : String , count : Long ) 
 
 Usage: 
 %scala 
 import d3._ 
 graphs.force ( 
   height = 500 , 
   width = 500 , 
   clicks : Dataset [ Edge ])

#### Visualize On-time and Early Arrivals

In [65]:
%scala
// On-time and Early Arrivals
import d3a._
graphs.force(
  height = 800,
  width = 1200,
  clicks = sql("""select src, dst as dest, count(1) as count from departureDelays_geo where delay <= 0 group by src, dst""").as[Edge])

<!DOCTYPE html>

#### Visualize Delayed Trips Departing from the West Coast

Notice that most of the delayed trips are with Western US cities

In [67]:
%scala
// Delayed Trips from CA, OR, and/or WA
import d3a._
graphs.force(
  height = 800,
  width = 1200,
  clicks = sql("""select src, dst as dest, count(1) as count from departureDelays_geo where state_src in ('CA', 'OR', 'WA') and delay > 0 group by src, dst""").as[Edge])

<!DOCTYPE html>

#### Visualize All Flights (from this dataset)

In [69]:
%scala
// Trips (from DepartureDelays Dataset)
import d3a._
graphs.force(
  height = 800,
  width = 1200,
  clicks = sql("""select src, dst as dest, count(1) as count from departureDelays_geo group by src, dst""").as[Edge])

<!DOCTYPE html>